In [1]:
import ultralytics
ultralytics.checks()

Ultralytics 8.3.223 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (13th Gen Intel Core i5-1335U)
Setup complete ✅ (12 CPUs, 15.3 GB RAM, 58.5/232.6 GB disk)


In [ ]:
# ai_client_webcam.py
import cv2
import requests
from ultralytics import YOLO
import time

# -----------------------------
# 1) 설정값
# -----------------------------
MODEL_PATH = '/home/vit/dev_ws/project/Media-Project/ai_server/12_model.pt'  # 너의 YOLOv8 가중치 파일 경로
DATA_SERVER_URL = "http://192.168.0.104:5002/process_trash"  # 데이터 서버 주소로 수정!

CAM_INDEX = 2          # 웹캠 번호 (기본 0)
CONF_THRES = 0.50      # 최소 신뢰도

# YOLO 클래스 인덱스 -> 실제 라벨 이름 (학습 시 사용한 순서)
CLASS_NAMES = {
    0: "종이",
    1: "종이팩",
    2: "종이컵",
    3: "캔류",
    4: "유리병",
    5: "페트",
    6: "플라스틱",
    7: "비닐",
    8: "유리+다중포장재",
    9: "페트+다중포장재",
    10: "스티로폼",
    11: "건전지",
}

# YOLO 클래스 idx -> 데이터서버의 type_id 매핑
#   5, 6, 9 -> 플라스틱(type_id=2)
#   3       -> 캔(type_id=3)
#   나머지 -> 일반(type_id=1)
YOLO_TO_TYPE_ID = {
    0: 1,   # 종이 -> 일반
    1: 1,   # 종이팩 -> 일반
    2: 1,   # 종이컵 -> 일반
    3: 3,   # 캔류 -> 캔
    4: 1,   # 유리병 -> 일반
    5: 2,   # 페트 -> 플라스틱
    6: 2,   # 플라스틱 -> 플라스틱
    7: 1,   # 비닐 -> 일반
    8: 1,   # 유리+다중포장재 -> 일반
    9: 2,   # 페트+다중포장재 -> 플라스틱
    10: 1,  # 스티로폼 -> 일반
    11: 1,  # 건전지 -> 일반
}

# -----------------------------
# 2) YOLO 결과에서 클래스 하나 뽑기 (cls/ det 둘 다 대응)
# -----------------------------
def get_pred_label(results):
    """YOLOv8 결과에서 가장 확신 높은 클래스 idx와 confidence 반환"""
    r = results[0]

    # (1) 분류 모델(cls)인 경우
    if hasattr(r, "probs") and r.probs is not None:
        idx = int(r.probs.top1)
        conf = float(r.probs.top1conf)
        return idx, conf

    # (2) 탐지 모델(det)인 경우: 가장 conf 높은 박스 하나 선택
    if hasattr(r, "boxes") and r.boxes is not None and len(r.boxes) > 0:
        max_conf_idx = int(r.boxes.conf.argmax().item())
        idx = int(r.boxes.cls[max_conf_idx].item())
        conf = float(r.boxes.conf[max_conf_idx].item())
        return idx, conf

    return None, None

# -----------------------------
# 3) 데이터 서버 호출
# -----------------------------
def send_to_data_server(type_id: int):
    """데이터 서버 /process_trash 에 type_id 전송"""
    payload = {"type_id": type_id}
    try:
        print(f"[INFO] 데이터서버로 전송: {payload}")
        resp = requests.post(DATA_SERVER_URL, json=payload, timeout=5)
        print(f"[INFO] 응답 코드: {resp.status_code}, 응답 내용: {resp.text}")
    except Exception as e:
        print(f"[ERROR] 데이터서버 통신 실패: {e}")

# -----------------------------
# 4) 메인 루프: 웹캠 + 키보드 캡처
# -----------------------------
def main():
    print("[INFO] YOLO 모델 로딩 중...")
    model = YOLO(MODEL_PATH)
    print("[INFO] 모델 로딩 완료!")

    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("[ERROR] 웹캠을 열 수 없습니다.")
        return

    print("[INFO] 웹캠 시작!")
    print(" - 'c' 또는 스페이스바: 현재 화면 캡처 후 분류 + 데이터서버 전송")
    print(" - 'q' : 종료")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[ERROR] 프레임을 읽을 수 없습니다.")
            break

        # 화면에 안내 문구 표시
        cv2.putText(frame, "Press 'c' or SPACE to capture, 'q' to quit",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imshow("DeepCycle AI Webcam", frame)
        key = cv2.waitKey(1) & 0xFF

        if key in [ord('c'), 32]:  # 'c' 또는 Space
            print("\n[INFO] ----- 캡처 & 분류 시작 -----")

            # YOLO 추론
            t0 = time.time()
            results = model.predict(source=frame, conf=CONF_THRES, verbose=False)
            infer_ms = (time.time() - t0) * 1000.0

            idx, conf = get_pred_label(results)
            if idx is None:
                print("[WARN] 객체를 인식하지 못했습니다.")
                continue

            label_kor = CLASS_NAMES.get(idx, f"cls_{idx}")
            type_id = YOLO_TO_TYPE_ID.get(idx, 1)  # 혹시 누락되면 기본 1(일반)

            print(
                f"[RESULT] YOLO idx={idx} "
                f"({label_kor}), type_id={type_id}, "
                f"conf={conf:.3f}, time={infer_ms:.1f}ms"
            )

            # 데이터 서버로 전송
            send_to_data_server(type_id)

        elif key == ord('q'):
            print("[INFO] 종료합니다.")
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


[INFO] YOLO 모델 로딩 중...
[INFO] 모델 로딩 완료!
[INFO] 웹캠 시작!
 - 'c' 또는 스페이스바: 현재 화면 캡처 후 분류 + 데이터서버 전송
 - 'q' : 종료


qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/vit/venv/eda_venv/lib/python3.12/site-packages/cv2/qt/plugins"



[INFO] ----- 캡처 & 분류 시작 -----
[RESULT] YOLO idx=3 (캔류), type_id=3, conf=0.818, time=441.0ms
[INFO] 데이터서버로 전송: {'type_id': 3}
[ERROR] 데이터서버 통신 실패: HTTPConnectionPool(host='192.168.0.104', port=5002): Max retries exceeded with url: /process_trash (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x756ca31a31d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

[INFO] ----- 캡처 & 분류 시작 -----
[RESULT] YOLO idx=3 (캔류), type_id=3, conf=0.824, time=323.0ms
[INFO] 데이터서버로 전송: {'type_id': 3}
[ERROR] 데이터서버 통신 실패: HTTPConnectionPool(host='192.168.0.104', port=5002): Max retries exceeded with url: /process_trash (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x756ca28b6480>: Failed to establish a new connection: [Errno 111] Connection refused'))

[INFO] ----- 캡처 & 분류 시작 -----
[RESULT] YOLO idx=3 (캔류), type_id=3, conf=0.732, time=313.5ms
[INFO] 데이터서버로 전송: {'type_id': 3}
[ERROR] 데이터서버 통신 실패: HTTPConnectionPool(host='192.16